In [6]:
# Import relevent libraries 
import os
import psycopg2
import pandas as pd
from psycopg2 import Error
import pandas.io.sql as sqlio

# Read in the data file that we need to insert
df = pd.read_excel("/Users/tahir/Desktop/Github/documents/db/Data/Cleaned_Root_letters.xlsx")

## THE CELL BELOW SHOULD CREATE 3 TABLES AND INSERT DATA IN THEM ## 


(SAME as the .py script)

In [52]:
# Import relevent libraries 
import os
import psycopg2
import pandas as pd
from psycopg2 import Error

# Read in the data file that we need to insert
df = pd.read_excel("./Data/Cleaned_Root_letters.xlsx")

# Clean the data right at the start 
df = df.convert_dtypes()
df['ARABIC'] = df['ARABIC'].str.strip()
df['Root_Letters'] = df['Root_Letters'].str.strip()

# Prep Data for RootWord Table 
lst = list(df['Root_Letters'])
lst = set(lst)

# Make it into a dataframe, and give it an index / ID column 
df_roots = pd.DataFrame(lst, columns=['Root_Words'])
df_roots = df_roots.reset_index()

# Turn it into string and get rid of white spaces 
df_roots = df_roots.convert_dtypes()
df_roots['Root_Words'] = df_roots['Root_Words'].str.strip()

# Put the data in the format the db accepts
root_data = []
for row in df_roots.index:
    arabic_text = df_roots.loc[row,"Root_Words"]
    unique_id = row
    root_data.append((row, arabic_text))

# Prep Data for the ArabicWord Table
arabic_df = df.drop(['ID', 'Transliteration'], axis=1)
arabic_df = arabic_df.drop_duplicates()
arabic_df["primary_key"] = list(range(2000,19623)) 
arabic_df = arabic_df.convert_dtypes()

# Merge the df to get the RootWords PK (called "index")
arabic_words_df = arabic_df.merge(df_roots, how='inner', left_on='Root_Letters', right_on='Root_Words')
arabic_words_df = arabic_words_df.drop(['Root_Letters', 'Root_Words'], axis=1)

# Rename the index column to RootID to make it clear
arabic_words_df.rename(columns = {'index':'RootID'}, inplace = True)

# Put the dataframe into the format for postgres sql insertion 
arabic_words_data = []
for index, row in arabic_words_df.iterrows():
    arabic_words_data.append( (row['primary_key'], row['ARABIC'], row['RootID']) )
    

In [53]:
    
try:         
    # Define DB connection parameters 
    dbHost = '127.0.0.1'
    dbPort = 5434
    dbUser = 'qj'
    dbPassword= 'Yatathakar123!'
    dbName = 'quranJourney'

    # connect to the PostgreSQL database
    connection = psycopg2.connect(user=dbUser,
                              password=dbPassword,
                              host=dbHost,
                              port=dbPort,
                              database=dbName)
    cursor = connection.cursor()

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
    


In [93]:
# Create function to get the quran text data 
def get_quran_text_df():
    """ Get the quran text dataframe """
    postgreSQL_select_Query = "select * from quran_text"
    cursor.execute(postgreSQL_select_Query)
    print("Selecting rows from table using cursor.fetchall")
    sql_table_result = cursor.fetchall()
    quran_text_df = pd.DataFrame(sql_table_result, columns=['index', 'sura', 'aya','text'])
    return quran_text_df


In [55]:

# Create functions to insert root words, arabic words and text to word data   
def insert_root_words(root_words_data):
    """ Insert Root Words Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO RootWord (RootID, RootWord) VALUES (%s ,%s)"""
    counter = 0
    for row in root_words_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into rootWord table")
    return counter

def insert_arabic_text(arabic_text_data):
    """ Insert Arabic Text Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO ArabicWord (WordID, Word, RootID) VALUES (%s ,%s, %s)"""
    counter = 0
    for row in arabic_text_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into ArabicWord table")
    return counter


In [94]:
quran_text_dataframe = get_quran_text_df()

Selecting rows from table using cursor.fetchall


In [102]:
quran_text_dataframe.to_excel('./Data/full_Quran.xlsx')

In [96]:
newdf= quran_text_dataframe.loc[quran_text_dataframe['sura'] == 114]

In [99]:
lst = newdf['text'].to_list()

In [100]:
lst

['بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ قُلْ أَعُوذُ بِرَبِّ النَّاسِ',
 'مَلِكِ النَّاسِ',
 'إِلَـٰهِ النَّاسِ',
 'مِن شَرِّ الْوَسْوَاسِ الْخَنَّاسِ',
 'الَّذِي يُوَسْوِسُ فِي صُدُورِ النَّاسِ',
 'مِنَ الْجِنَّةِ وَالنَّاسِ']

In [67]:
df10.drop(columns = ['ID', 'Transliteration', 'Surah', 'aua')

,ID,ARABIC,Transliteration,Root_Letters,Surah,Ayah,index,sura,aya,text
0,1:1,بِسْمِ,bis'mi,س م و,1,1,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ
1,1:1,بِسْمِ,bis'mi,س م و,1,1,2,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ
2,1:1,بِسْمِ,bis'mi,س م و,1,1,3,1,3,الرَّحْمَـٰنِ الرَّحِيمِ
3,1:1,بِسْمِ,bis'mi,س م و,1,1,4,1,4,مَالِكِ يَوْمِ الدِّينِ
4,1:1,بِسْمِ,bis'mi,س م و,1,1,5,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ
...,...,...,...,...,...,...,...,...,...,...
9156873,114:6,وَالنَّاسِ,wal-nāsi,ن و س,114,6,6232,114,2,مَلِكِ النَّاسِ
9156874,114:6,وَالنَّاسِ,wal-nāsi,ن و س,114,6,6233,114,3,إِلَـٰهِ النَّاسِ
9156875,114:6,وَالنَّاسِ,wal-nāsi,ن و س,114,6,6234,114,4,مِن شَرِّ الْوَسْوَاسِ الْخَنَّاسِ
9156876,114:6,وَالنَّاسِ,wal-nāsi,ن و س,114,6,6235,114,5,الَّذِي يُوَسْوِسُ فِي صُدُورِ النَّاسِ


In [68]:
quran_text_dataframe

,index,sura,aya,text
0,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ
1,2,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ
2,3,1,3,الرَّحْمَـٰنِ الرَّحِيمِ
3,4,1,4,مَالِكِ يَوْمِ الدِّينِ
4,5,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ
...,...,...,...,...
6231,6232,114,2,مَلِكِ النَّاسِ
6232,6233,114,3,إِلَـٰهِ النَّاسِ
6233,6234,114,4,مِن شَرِّ الْوَسْوَاسِ الْخَنَّاسِ
6234,6235,114,5,الَّذِي يُوَسْوِسُ فِي صُدُورِ النَّاسِ


In [84]:
df

,ID,ARABIC,Transliteration,Root_Letters,Surah,Ayah
0,1:1,بِسْمِ,bis'mi,س م و,1,1
1,1:1,اللَّهِ,l-lahi,ا ل ه,1,1
2,1:1,الرَّحْمَنِ,l-raḥmāni,ر ح م,1,1
3,1:1,الرَّحِيمِ,l-raḥīmi,ر ح م,1,1
4,1:2,الْحَمْدُ,al-ḥamdu,ح م د,1,2
...,...,...,...,...,...,...
77426,114:5,صُدُورِ,ṣudūri,ص د ر,114,5
77427,114:5,النَّاسِ,l-nāsi,ن و س,114,5
77428,114:6,مِنَ,mina,م ن,114,6
77429,114:6,الْجِنَّةِ,l-jinati,ج ن ن,114,6


In [78]:
index_id = []
ayahs = df['Ayah'].to_list()

In [81]:
index_id = []
count = 0
for i in range(len(ayahs)):
    if ayahs[i] != ayahs[i-1]:
        count+=1 
        index_id.append(count)
    else:
        index_id.append(count)
    

In [85]:
df['index_id'] = index_id

In [88]:
df

,ID,ARABIC,Transliteration,Root_Letters,Surah,Ayah,index_id
0,1:1,بِسْمِ,bis'mi,س م و,1,1,1
1,1:1,اللَّهِ,l-lahi,ا ل ه,1,1,1
2,1:1,الرَّحْمَنِ,l-raḥmāni,ر ح م,1,1,1
3,1:1,الرَّحِيمِ,l-raḥīmi,ر ح م,1,1,1
4,1:2,الْحَمْدُ,al-ḥamdu,ح م د,1,2,2
...,...,...,...,...,...,...,...
77426,114:5,صُدُورِ,ṣudūri,ص د ر,114,5,6235
77427,114:5,النَّاسِ,l-nāsi,ن و س,114,5,6235
77428,114:6,مِنَ,mina,م ن,114,6,6236
77429,114:6,الْجِنَّةِ,l-jinati,ج ن ن,114,6,6236


In [87]:
6,236

(6, 236)

In [90]:
quran_text_dataframe['sura'] = 2

In [91]:
quran_text_dataframe

,index,sura,aya,text
0,1,2,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ
1,2,2,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ
2,3,2,3,الرَّحْمَـٰنِ الرَّحِيمِ
3,4,2,4,مَالِكِ يَوْمِ الدِّينِ
4,5,2,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ
...,...,...,...,...
6231,6232,2,2,مَلِكِ النَّاسِ
6232,6233,2,3,إِلَـٰهِ النَّاسِ
6233,6234,2,4,مِن شَرِّ الْوَسْوَاسِ الْخَنَّاسِ
6234,6235,2,5,الَّذِي يُوَسْوِسُ فِي صُدُورِ النَّاسِ


In [76]:
df.to_excel("./Data/TextToWords.xlsx")  

In [71]:
# Get the IndexIDs from the Quran Table  -- need to merge it since different lengths
df11 =  df.merge(quran_text_dataframe, left_on ="Ayah", right_on = "aya", how = 'inner')
df11

,ID,ARABIC,Transliteration,Root_Letters,Surah,Ayah,index,sura,aya,text
0,1:1,بِسْمِ,bis'mi,س م و,1,1,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ
1,1:1,بِسْمِ,bis'mi,س م و,1,1,8,2,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ الم
2,1:1,بِسْمِ,bis'mi,س م و,1,1,294,3,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ الم
3,1:1,بِسْمِ,bis'mi,س م و,1,1,494,4,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ يَا أَ...
4,1:1,بِسْمِ,bis'mi,س م و,1,1,670,5,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ يَا أَ...
...,...,...,...,...,...,...,...,...,...,...
4130544,2:286,مَوْلَانَا,mawlānā,و ل ي,2,286,293,2,286,لَا يُكَلِّفُ اللَّهُ نَفْسًا إِلَّا وُسْعَهَا...
4130545,2:286,فَانْصُرْنَا,fa-unṣur'nā,ن ص ر,2,286,293,2,286,لَا يُكَلِّفُ اللَّهُ نَفْسًا إِلَّا وُسْعَهَا...
4130546,2:286,عَلَى,ʿalā,ع ل ي,2,286,293,2,286,لَا يُكَلِّفُ اللَّهُ نَفْسًا إِلَّا وُسْعَهَا...
4130547,2:286,الْقَوْمِ,l-qawmi,ق و م,2,286,293,2,286,لَا يُكَلِّفُ اللَّهُ نَفْسًا إِلَّا وُسْعَهَا...


In [57]:
 #Get the Quran Text Table 
quran_text_dataframe = get_quran_text_df()

# Create surah and ayah columns 
surah, ayah = [], []
for row in df["ID"]:
    surah.append( row.split(":")[0] ) 
    ayah.append( row.split(":")[1] ) 
df["Surah"] = surah
df["Ayah"] = ayah
df["Surah"] = df["Surah"].values.astype(int)
df["Ayah"] = df["Ayah"].values.astype(int)

# Get the IndexIDs from the Quran Table  -- need to merge it since different lengths
df10 =  df.merge(quran_text_dataframe, left_on ="Surah", right_on = "sura", how = 'inner')

# Generate our text to words table
TextToWords_df = df10.merge(arabic_words_df, left_on ="ARABIC", right_on = "ARABIC", how = 'inner')
TextToWords_df = TextToWords_df.reset_index()

# Put the dataframe into the format for postgres sql insertion 
TexttoWord = []

new_df = TextToWords_df.drop_duplicates(
  subset = ['index', 'primary_key'],
  keep = 'last').reset_index(drop = True)
  
# print latest dataframe
for index, row in new_df.iterrows():
    tup = tuple((row['index'], row['primary_key']))
    TexttoWord.append( tup )


Selecting rows from table using cursor.fetchall


In [59]:
df10.head()

,ID,ARABIC,Transliteration,Root_Letters,Surah,Ayah,index,sura,aya,text
0,1:1,بِسْمِ,bis'mi,س م و,1,1,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ
1,1:1,بِسْمِ,bis'mi,س م و,1,1,2,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ
2,1:1,بِسْمِ,bis'mi,س م و,1,1,3,1,3,الرَّحْمَـٰنِ الرَّحِيمِ
3,1:1,بِسْمِ,bis'mi,س م و,1,1,4,1,4,مَالِكِ يَوْمِ الدِّينِ
4,1:1,بِسْمِ,bis'mi,س م و,1,1,5,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ


In [61]:
len(TexttoWord)

4834957

In [64]:
TextToWords_df = TextToWords_df.drop_duplicates()

In [65]:
TextToWords_df

,level_0,ID,ARABIC,Transliteration,Root_Letters,Surah,Ayah,index,sura,aya,text,primary_key,RootID
0,0,1:1,بِسْمِ,bis'mi,س م و,1,1,1,1,1,بِسْمِ اللَّهِ الرَّحْمَـٰنِ الرَّحِيمِ,2000,917
1,1,1:1,بِسْمِ,bis'mi,س م و,1,1,2,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ,2000,917
2,2,1:1,بِسْمِ,bis'mi,س م و,1,1,3,1,3,الرَّحْمَـٰنِ الرَّحِيمِ,2000,917
3,3,1:1,بِسْمِ,bis'mi,س م و,1,1,4,1,4,مَالِكِ يَوْمِ الدِّينِ,2000,917
4,4,1:1,بِسْمِ,bis'mi,س م و,1,1,5,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ,2000,917
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9199417,9199417,114:5,يُوَسْوِسُ,yuwaswisu,و س و س,114,5,6232,114,2,مَلِكِ النَّاسِ,19622,960
9199418,9199418,114:5,يُوَسْوِسُ,yuwaswisu,و س و س,114,5,6233,114,3,إِلَـٰهِ النَّاسِ,19622,960
9199419,9199419,114:5,يُوَسْوِسُ,yuwaswisu,و س و س,114,5,6234,114,4,مِن شَرِّ الْوَسْوَاسِ الْخَنَّاسِ,19622,960
9199420,9199420,114:5,يُوَسْوِسُ,yuwaswisu,و س و س,114,5,6235,114,5,الَّذِي يُوَسْوِسُ فِي صُدُورِ النَّاسِ,19622,960


In [51]:
# Import relevent libraries 
import os
import psycopg2
import pandas as pd
from psycopg2 import Error

# Read in the data file that we need to insert
df = pd.read_excel("./Data/Cleaned_Root_letters.xlsx")

# Clean the data right at the start 
df = df.convert_dtypes()
df['ARABIC'] = df['ARABIC'].str.strip()
df['Root_Letters'] = df['Root_Letters'].str.strip()

# Prep Data for RootWord Table 
lst = list(df['Root_Letters'])
lst = set(lst)

# Make it into a dataframe, and give it an index / ID column 
df_roots = pd.DataFrame(lst, columns=['Root_Words'])
df_roots = df_roots.reset_index()

# Turn it into string and get rid of white spaces 
df_roots = df_roots.convert_dtypes()
df_roots['Root_Words'] = df_roots['Root_Words'].str.strip()

# Put the data in the format the db accepts
root_data = []
for row in df_roots.index:
    arabic_text = df_roots.loc[row,"Root_Words"]
    unique_id = row
    root_data.append((row, arabic_text))

# Prep Data for the ArabicWord Table
arabic_df = df.drop(['ID', 'Transliteration'], axis=1)
arabic_df = arabic_df.drop_duplicates()
arabic_df["primary_key"] = list(range(2000,19623)) 
arabic_df = arabic_df.convert_dtypes()

# Merge the df to get the RootWords PK (called "index")
arabic_words_df = arabic_df.merge(df_roots, how='inner', left_on='Root_Letters', right_on='Root_Words')
arabic_words_df = arabic_words_df.drop(['Root_Letters', 'Root_Words'], axis=1)

# Rename the index column to RootID to make it clear
arabic_words_df.rename(columns = {'index':'RootID'}, inplace = True)

# Put the dataframe into the format for postgres sql insertion 
arabic_words_data = []
for index, row in arabic_words_df.iterrows():
    arabic_words_data.append( (row['primary_key'], row['ARABIC'], row['RootID']) )
    
try:         
    # Define DB connection parameters 
    dbHost = '127.0.0.1'
    dbPort = 5434
    dbUser = 'qj'
    dbPassword= 'Yatathakar123!'
    dbName = 'quranJourney'

    # connect to the PostgreSQL database
    connection = psycopg2.connect(user=dbUser,
                              password=dbPassword,
                              host=dbHost,
                              port=dbPort,
                              database=dbName)
    cursor = connection.cursor()

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
    

# Create functions to insert root words, arabic words and text to word data   
def insert_root_words(root_words_data):
    """ Insert Root Words Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO RootWord (RootID, RootWord) VALUES (%s ,%s)"""
    counter = 0
    for row in root_words_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into rootWord table")
    return counter

def insert_arabic_text(arabic_text_data):
    """ Insert Arabic Text Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO ArabicWord (WordID, Word, RootID) VALUES (%s ,%s, %s)"""
    counter = 0
    for row in arabic_text_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into ArabicWord table")
    return counter

# Create function to get the quran text data 
def get_quran_text_df():
    """ Get the quran text dataframe """
    postgreSQL_select_Query = "select * from quran_text"
    cursor.execute(postgreSQL_select_Query)
    print("Selecting rows from table using cursor.fetchall")
    sql_table_result = cursor.fetchall()
    quran_text_df = pd.DataFrame(sql_table_result, columns=['index', 'sura', 'aya','text'])
    return quran_text_df

########################################
### GET data for  TEXT TO WORD table ###
########################################

# Insert the root_words data 
insert_root_words(root_data)

# Insert the arabic words data 
insert_arabic_text(arabic_words_data)

# Get the Quran Text Table 
quran_text_dataframe = get_quran_text_df()

# Create surah and ayah columns 
surah, ayah = [], []
for row in df["ID"]:
    surah.append( row.split(":")[0] ) 
    ayah.append( row.split(":")[1] ) 
df["Surah"] = surah
df["Ayah"] = ayah
df["Surah"] = df["Surah"].values.astype(int)
df["Ayah"] = df["Ayah"].values.astype(int)

# Get the IndexIDs from the Quran Table  -- need to merge it since different lengths
index_id, count = [], 0
ayahs = df['Ayah'].to_list()
# Essentially generating each ayah = 1 unique id, from 1 - 6000. 
for i in range(len(ayahs)):
    if ayahs[i] != ayahs[i-1]:
        count+=1 
        index_id.append(count)
    else:
        index_id.append(count)
    
# Generate our text to words table
TextToWords_df = df.merge(arabic_words_df, left_on ="ARABIC", right_on = "ARABIC", how = 'inner')
TextToWords_df = TextToWords_df.reset_index()

# Put the dataframe into the format for postgres sql insertion 
TexttoWord = []

new_df = TextToWords_df.drop_duplicates(
  subset = ['index', 'primary_key'],
  keep = 'last').reset_index(drop = True)
  
# print latest dataframe
for index, row in new_df.iterrows():
    tup = tuple((row['index'], row['primary_key']))
    TexttoWord.append( tup )

def insert_text_to_word(text_to_word_data):
    """ Insert Text to Word  Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO TextToWord (AyahID, WordID) VALUES (%s, %s)"""
    counter = 0
    for row in text_to_word_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into TextToWord table")
    return counter

# Insert text_to_word_df
insert_text_to_word(TexttoWord)

# Close the DB connection 
cursor.close()
connection.close()
print("PostgreSQL connection is closed")

UniqueViolation: duplicate key value violates unique constraint "rootword_pkey"
DETAIL:  Key (rootid)=(0) already exists.


In [ ]:
# Connect to the Postgres SQL Database 
try:         
    # Define DB connection parameters 
    dbHost = '127.0.0.1'
    dbPort = 5434
    dbUser = 'qj'
    dbPassword= 'Yatathakar123!'
    dbName = 'quranJourney'

    # connect to the PostgreSQL database
    connection = psycopg2.connect(user=dbUser,
                              password=dbPassword,
                              host=dbHost,
                              port=dbPort,
                              database=dbName)
    cursor = connection.cursor()

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

In [ ]:
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cursor.fetchall():
    print(table)

In [ ]:
postgreSQL_select_Query = "select * from quran_text"
cursor.execute(postgreSQL_select_Query)
print("Selecting rows from table using cursor.fetchall")
sql_table_result = cursor.fetchall()

In [ ]:
sql_table_result[:10]

In [ ]:
# Create surah and ayah columns 
surah, ayah = [], []
for row in df["ID"]:
    surah.append( row.split(":")[0] ) 
    ayah.append( row.split(":")[1] ) 
df["Surah"] = surah
df["Ayah"] = ayah
df["Surah"] = df["Surah"].values.astype(int)
df["Ayah"] = df["Ayah"].values.astype(int)

# Get the IndexIDs from the Quran Table  -- need to merge it since different lengths
df10 =  df.merge(quran_text_dataframe, left_on ="Surah", right_on = "sura", how = 'inner')

# Generate our text to words table
TextToWords_df = df.merge(arabic_words_df, left_on ="ARABIC", right_on = "ARABIC", how = 'inner')
TextToWords_df = TextToWords_df.reset_index()

# Put the dataframe into the format for postgres sql insertion 
TexttoWord = []
for index, row in TextToWords_df.iterrows():
    tup = tuple((row['index'], row['primary_key']))
    if tup not in TexttoWord:
        TexttoWord.append( tup )
    else:
        pass

In [ ]:
df10

In [ ]:

# Generate our text to words table
TextToWords_df = df.merge(arabic_words_df, left_on ="ARABIC", right_on = "ARABIC", how = 'inner')
TextToWords_df = TextToWords_df.reset_index()

# Put the dataframe into the format for postgres sql insertion 
TexttoWord = []
for index, row in TextToWords_df.iterrows():
    tup = tuple((row['Ayah'], row['primary_key']))
    if tup not in TexttoWord:
        TexttoWord.append( tup )
    else:
        pass

In [ ]:
quran_text_dataframe.head()

In [ ]:
TexttoWord

In [ ]:
quran_text = pd.DataFrame(sql_table_result, columns=['index', 'sura', 'aya','text'])
quran_text.head()
# print(df)

In [ ]:
# Close the DB connection 
cursor.close()
connection.close()

# List out all tables we have

In [ ]:
# Create another column for surah and ayat, extracted from ID
surah, ayah = [], []
for row in df["ID"]:
    surah.append( row.split(":")[0] ) 
    ayah.append( row.split(":")[1] ) 

df["Surah"] = surah
df["Ayah"] = ayah

df["Surah"] = df["Surah"].values.astype(int)
df["Ayah"] = df["Ayah"].values.astype(int)
df

In [ ]:
df_roots

In [ ]:
arabic_words_df

In [ ]:
quran_text

# Create the TexttoWord table

In [ ]:
temp = quran_text

In [ ]:
TextToWords_df = df.merge(arabic_words_df, left_on ="ARABIC", right_on = "ARABIC", how = 'inner')
TextToWords_df = TextToWords_df.reset_index()

# Put the dataframe into the format for postgres sql insertion 
TexttoWord = []
for index, row in TextToWords_df.iterrows():
    TexttoWord.append( (row['index'], row['Ayah'], row['primary_key']))
    
def insert_text_to_word(text_to_word_data):
    """ Insert Text to Word  Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO TextToWord (IndexID, AyahID, WordID) VALUES (%s, %s ,%s)"""
    counter = 0
    for row in text_to_word_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into TextToWord table")
    return counter

insert_text_to_word(TexttoWord)


In [ ]:
temp2 = temp2.reset_index()
temp2

In [ ]:
# Put the dataframe into the format for postgres sql insertion 
TexttoWord = []
for index, row in temp2.iterrows():
    TexttoWord.append( (row['index'], row['Ayah'], row['primary_key']))

In [ ]:
def insert_text_to_word(text_to_word_data):
    """ Insert Text to Word  Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO TextToWord (IndexID, AyahID, WordID) VALUES (%s, %s ,%s)"""
    counter = 0
    for row in text_to_word_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into TextToWord table")
    return counter


In [ ]:
insert_text_to_word(TexttoWord)

In [ ]:
# Connect to the Postgres SQL Database 
try:         
    # Define DB connection parameters 
    dbHost = '127.0.0.1'
    dbPort = 5434
    dbUser = 'qj'
    dbPassword= 'Yatathakar123!'
    dbName = 'quranJourney'

    # connect to the PostgreSQL database
    connection = psycopg2.connect(user=dbUser,
                              password=dbPassword,
                              host=dbHost,
                              port=dbPort,
                              database=dbName)
    cursor = connection.cursor()

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)

In [ ]:
postgreSQL_select_Query = "select * from quran_text"
cursor.execute(postgreSQL_select_Query)
print("Selecting rows from quran table using cursor.fetchall")
quran_text_full = cursor.fetchall()

print("Print each row and it's columns values")
index, surah, ayah, text = [], [], [], [] 
for row in quran_text_full:
    index.append(row[0])
    surah.append(row[1])
    ayah.append(row[2])
    text.append(row[3])
    
quran_text_df = pd.DataFrame(
    {'Index': index,
     'Surah': surah,
     'Ayah': ayah,
     'Text': text
    })
quran_text_df.head(10)

In [ ]:
postgreSQL_select_Query = "select * from ArabicWord"
cursor.execute(postgreSQL_select_Query)
print("Selecting rows from quran table using cursor.fetchall")
quran_text_full = cursor.fetchall()
print(quran_text_full[:100])

In [ ]:
# Close the DB connection 
cursor.close()
connection.close()
print("PostgreSQL connection is closed")

# Now that we have the data... its time to prep it for our db

In [ ]:
df["Surah"] = df["Surah"].values.astype(int)
df["Ayah"] = df["Ayah"].values.astype(int)
df

In [ ]:
TextToWord = df.merge(quran_text_full, on = ["Word", "Ayah" ], how = 'inner')

In [ ]:
quran_text_df

In [ ]:
df["Surah"] = df["Surah"].values.astype(int)
df["Ayah"] = df["Ayah"].values.astype(int)

In [ ]:
df.dtypes

In [ ]:
quran_text_df

In [ ]:
df

In [ ]:
quran_text

In [ ]:
df3  = quran_text_df.merge(df, on = ["Surah", "Ayah" ], how = 'inner')

In [ ]:
df3

In [ ]:
df4 = df3.merge(arabic_words_df, on = "ARABIC", how = 'inner')
df4fa885769a2d0

In [ ]:
df4.head(30)

In [ ]:
df5 = df4.drop_duplicates()
df5.nunique()

In [ ]:
df5["index2"] = df5["Index"]
df5 = df5.drop(["Index"], axis=1)

In [ ]:
df5.index

In [ ]:
# Prep data
# Put the data in the format the db accepts
text_to_word_data = []
for row in df5.index:
    AyahID = text_to_word_data.loc[row,"index2"]
    WordID = text_to_word_data.loc[row,"primary_key"]
    
    text_to_word_data.append((AyahID, WordID))


In [ ]:
df4 = df3.merge(arabic_words_df, left_on ="ARABIC", right_on = "ARABIC", how = 'outer')
df4.head()

In [ ]:
df5 = df4.drop_duplicates()

In [ ]:
df5

In [ ]:
arabic_words_df

In [ ]:
AyahID = quran_text_df["Index"]
WordID = 

In [ ]:
# Prep Data for RootWords Table 
lst = list(df['Root_Letters'])
lst = set(lst)

# Make it into a dataframe, and give it an index / ID column 
df_roots = pd.DataFrame(lst, columns=['Root_Words'])
df_roots = df_roots.reset_index()

# df_roots["Root_Words"] = lst
df_roots["Root_Words"] = df_roots["Root_Words"].values.astype(str)

# Put the data in the format the db accepts
root_data = []
for row in df_roots.index:
    arabic_text = df_roots.loc[row,"Root_Words"]
    unique_id = row
    root_data.append((row, arabic_text))

# Prep Data for the ArabicWord Table
arabic_df = df.drop(['ID', 'Transliteration'], axis=1)
# arabic_df = arabic_df.drop_duplicates()
arabic_df["primary_key"] = list(range(2000,19623)) 

# Merge the df to get the RootWords PK (called "index")
arabic_words_df = arabic_df.merge(df_roots, how='inner', left_on='Root_Letters', right_on='Root_Words')
arabic_words_df = arabic_words_df.drop(['Root_Letters', 'Root_Words'], axis=1)

# Rename the index column to RootID to make it clear
arabic_words_df.rename(columns = {'index':'RootID'}, inplace = True)

# Put the dataframe into the format for postgres sql insertion 
arabic_words_data = []
for index, row in arabic_words_df.iterrows():
    arabic_words_data.append( (row['primary_key'], row['ARABIC'], row['RootID']) )

In [ ]:
arabic_words_df

In [ ]:
df_roots

In [ ]:
# Close the DB connection 
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")

In [ ]:
# Import relevent libraries 
import os
import psycopg2
import pandas as pd
from psycopg2 import Error

# Read in the data file that we need to insert
df = pd.read_excel("/Users/tahir/Desktop/Github/documents/roots/Data/Cleaned_Root_letters.xlsx")

# Prep Data for RootWords Table 
lst = list(df['Root_Letters'])
lst = set(lst)

# Make it into a dataframe, and give it an index / ID column 
df_roots = pd.DataFrame(lst, columns=['Root_Words'])
df_roots = df_roots.reset_index()

# df_roots["Root_Words"] = lst
df_roots["Root_Words"] = df_roots["Root_Words"].values.astype(str)

# Put the data in the format the db accepts
root_data = []
for row in df_roots.index:
    arabic_text = df_roots.loc[row,"Root_Words"]
    unique_id = row
    root_data.append((row, arabic_text))

# Prep Data for the ArabicWord Table
arabic_df = df.drop(['ID', 'Transliteration'], axis=1)
arabic_df = arabic_df.drop_duplicates()
arabic_df["primary_key"] = list(range(2000,19623)) 

# Merge the df to get the RootWords PK (called "index")
arabic_words_df = arabic_df.merge(df_roots, how='inner', left_on='Root_Letters', right_on='Root_Words')
arabic_words_df = arabic_words_df.drop(['Root_Letters', 'Root_Words'], axis=1)

# Rename the index column to RootID to make it clear
arabic_words_df.rename(columns = {'index':'RootID'}, inplace = True)

# Put the dataframe into the format for postgres sql insertion 
arabic_words_data = []
for index, row in arabic_words_df.iterrows():
    arabic_words_data.append( (row['primary_key'], row['ARABIC'], row['RootID']) )

# Connect to the Postgres SQL Database 
try:         
    # Define DB connection parameters 
    dbHost = '127.0.0.1'
    dbPort = 5434
    dbUser = 'qj'
    dbPassword= 'Yatathakar123!'
    dbName = 'quranJourney'

    # connect to the PostgreSQL database
    connection = psycopg2.connect(user=dbUser,
                              password=dbPassword,
                              host=dbHost,
                              port=dbPort,
                              database=dbName)
    cursor = connection.cursor()

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
    
# Create functions to insert root words and arabic words data     
def insert_root_words(root_words_data):
    """ Insert Root Words Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO RootWords (RootID, RootWord) VALUES (%s ,%s)"""
    counter = 0
    for row in root_words_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into rootWords table")
    return counter

def insert_arabic_text(arabic_text_data):
    """ Insert Arabic Text Data into the POSTGRES DB """
    postgres_insert_query = """ INSERT INTO ArabicWord (WordID, Word, RootID) VALUES (%s ,%s, %s)"""
    counter = 0
    for row in arabic_text_data:
        record_to_insert = row
        cursor.execute(postgres_insert_query, record_to_insert)
        connection.commit()
        counter += cursor.rowcount
    print(counter, "records inserted successfully into ArabicWord table")
    return counter

if __name__ == '__main__':

    # Insert the root_words data 
    #insert_root_words(root_data)

    # Insert the arabic words data 
    #insert_arabic_text(arabic_words_data)

    # Close the DB connection 
    cursor.close()
    connection.close()
    print("PostgreSQL connection is closed")